# Todo
Basic Stats
* alle Dialogdaten und alle Skripte in jeweils eine Datei

auf Basis der Dialogdaten:
* Sprecheranteile der Personen: mit groupby nach Personen -> Wörter zählen (zwischen Leerzeilen) -> Maxi
* Dialoge aller Personen extrahieren (Frodo, Sam, Merry, Pippin, Gandalf, Aragorn, Legolas, Gimli, Gollum, Saruman, Sauron) -> Marina
* Wordclouds für Personen erstellen -> Marina

auf Basis der Regieanweisungen und Dialogdaten
* zählen wie häufig die einzelnen Personen erwähnt werden -> Maxi

Einarbeitung in Sentiment Analysis

# Sentiment Analysis

In [2]:
from bs4 import BeautifulSoup
import requests
import os
import re
import pandas as pd
import numpy as np
from textblob import TextBlob
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

In [3]:
# load reduced data of film 1
df_dialog_film_1 = pd.read_csv("C:/Users/Marina/Documents/Studium/Master Mainz/MA Digitale Methodik/Ü NLP/Daten von James Tauber/LOTR Subtitles reduced film 1.csv", sep=";")
df_dialog_film_1

,ORIG ID,SPEAKER,SUBTITLE TEXT,SCENE,ITUNES CHAPTER
0,NaN,NaN,NaN,NaN,1.0
1,2,Galadriel VO,The world is changed.,NaN,NaN
2,4,Galadriel VO,I feel it in the water.,NaN,NaN
3,6,Galadriel VO,I feel it in the earth.,NaN,NaN
4,8,Galadriel VO,I smell it in the air.,NaN,NaN
...,...,...,...,...,...
1968,1779,Frodo,I don’t suppose we’ll ever see them again.,NaN,NaN
1969,1780,Sam,"We may yet, Mr. Frodo.",NaN,NaN
1970,1781,Sam,We may.,NaN,NaN
1971,1782,Frodo,Sam...,NaN,NaN


In [4]:
# load dialog data of script 2
df_dialog_film_2 = pd.read_csv("C:/Users/Marina/Documents/Studium/Master Mainz/MA Digitale Methodik/Ü NLP/Daten von James Tauber/LOTR Subtitles reduced film 2.csv", sep=";")
df_dialog_film_2

,ORG ID,SPEAKER,SUBTITLE TEXT,SCENE,ITUNES CHAPTER
0,NaN,NaN,NaN,NaN,1.0
1,1.0,Gandalf,You cannot pass!,NaN,NaN
2,1.0,Frodo,Gandalf!,NaN,NaN
3,2.0,Gandalf,"I am a servant of the Secret Fire, wielder of ...",NaN,NaN
4,3.0,Gandalf,Go back to the Shadow.,NaN,NaN
...,...,...,...,...,...
2191,2064.0,Gollum,And then we takes it once they're dead.,NaN,NaN
2192,2065.0,Smeagol,Once they're dead.,NaN,NaN
2193,2066.0,Smeagol,"Come on, Hobbits. Long ways to go yet.",NaN,NaN
2194,2067.0,Smeagol,Smeagol will show you the way.,NaN,NaN


In [5]:
# load dialog data of script 3
df_dialog_film_3 = pd.read_csv("C:/Users/Marina/Documents/Studium/Master Mainz/MA Digitale Methodik/Ü NLP/Daten von James Tauber/LOTR Subtitles reduced film 3.csv", sep=";")
df_dialog_film_3

,ORG ID,SPEAKER,SUBTITLE TEXT,SCENE,ITUNES CHAPTER
0,NaN,NaN,NaN,NaN,1.0
1,1.0,Deagol,Sméagol!,NaN,NaN
2,2.0,Deagol,I've got one!,NaN,NaN
3,3.0,Deagol,"I've got a fish, Smeag. Smeagol!",NaN,NaN
4,4.0,Smeagol,Pull it in. Go on. Go on. Go on. Pull it in.,NaN,NaN
...,...,...,...,...,...
1871,1811.0,Frodo,You will have to be one and whole for many years.,NaN,NaN
1872,1812.0,Frodo,You have so much to enjoy and to be and to do.,NaN,NaN
1873,1813.0,Frodo,Your part in the story will go on.,NaN,NaN
1874,1814.0,Sam,Well...,NaN,NaN


### Helper functions

#### Get complete dialog data of a single film

In [7]:
def getDialogOfFilm(dfFilm):
    film = dfFilm["SUBTITLE TEXT"]
    film_text = ""
    for text in film:
        if (not pd.isna(text)):
            film_text = f'{film_text} {text}'
            # replaces "... ..." or "..." (and variations with whitespace) with one single space character
            film_text = re.sub(r'\s?\.\.\.\s?\.?\.?\.?\s?', " ", film_text)
    return film_text

In [8]:
# dialog text of film 1
film_1_text = getDialogOfFilm(df_dialog_film_1)
film_1_text

" The world is changed. I feel it in the water. I feel it in the earth. I smell it in the air. Much that once was  is lost. For none now live who remember it. It began with the forging of the Great Rings. Three were given to the Elves: Immortal, wisest and fairest of all beings. Seven to the Dwarf-lords: Great miners and craftsmen of the mountain halls. And nine  nine rings were gifted to the race of Men  who, above all else, desire power. For within these rings was bound the strength and will to govern each race. But they were all of them deceived. For another ring was made. In the land of Mordor, in the fires of Mount Doom  the Dark Lord Sauron forged in secret a Master Ring  to control all others. And into this Ring he poured his cruelty, his malice  and his will to dominate all life. One Ring to rule them all. One by one  the Free Lands of Middle-earth fell to the power of the ring. But there were some who resisted. A Last Alliance of Men and Elves marched against the armies of Mor

#### Get text data of specific character

In [10]:
# takes the name of a character (exactly as spelled in the data frame) and returns all dialog of this character
def getTextOfCharacter(dfFilm, character):
    character = dfFilm[dfFilm["SPEAKER"] == character]
    character_text = ""
    for text in character["SUBTITLE TEXT"]:
        if (not pd.isna(text)):
            character_text = f'{character_text} {text}'
            # replaces "... ..." with space character
            character_text = re.sub(r'\s?\.\.\.\s?\.?\.?\.?\s?', " ", character_text)
    return character_text

In [11]:
# Frodo's dialog text
frodo = getTextOfCharacter(df_dialog_film_1, "Frodo")
frodo

" You’re late. It’s wonderful to see you, Gandalf! You know Bilbo. He’s got the whole piece in an uproar. Half the Shire’s been invited. And the rest of them are turning up anyway. To tell you the truth, Bilbo’s been a bit odd lateIy. I mean, more than usual. He’s taken to locking himself in his study. He spends hours and hours poring over old maps when he thinks I’m not looking. He’s up to something. All right, then. Keep your secrets. But I know you have something to do with it. Before you came along, we Bagginses were very well thought of. Never had any adventures or did anything unexpected. Whatever you did, you’ve been officially labeled a disturber of the peace. Gandalf? I’m glad you’re back. Go on, Sam. Ask Rosie for a dance. Oh, no, you don’t. Go on! Bilbo? Bilbo, have you been at the Gaffer’s home brew? Bilbo. Bilbo, watch out for the dragon! Speech! Bilbo! Bilbo! He’s gone, hasn’t he? He talked for so long about leaving  I didn’t think he’d really do it. Gandalf? Where are yo

In [12]:
# Gandalf's dialog text
gandalf = getTextOfCharacter(df_dialog_film_1, "Gandalf")
gandalf

' Down from the door where it began And I must follow if I can The road goes ever on and on Down from the door where it began Now far ahead the road has gone And I must follow if I can A wizard is never late, Frodo Baggins. Nor is he early. He arrives precisely when he means to. You didn’t think I’d miss your Uncle Bilbo’s birthday? So how is the old rascal? I hear it’s going to be a party of special magnificence. Well, that should please him. What? Good gracious me. Indeed. If you’re referring to the incident with the dragon. I was barely involved. All I did was give your uncle a little nudge out of the door. Oh, really? So am I, dear boy. So am I. Whoa, whoa. And what about very old friends? Bilbo Baggins. Good to see you. 111 years old! Who would believe it? You haven’t aged a day. Just tea, thank you. Just tea, thank you. No, not at all. So you mean to go through with your plan? Frodo suspects something. You will tell him, won’t you? He’s very fond of you. Whoa Up they go! Meriadoc

## First Sentiment Analysis test using TextBlob

Info:
* Video: [Sentiment Analysis using Textblob](https://www.youtube.com/watch?v=6bg-TNoT5_Y)
* Text Tutorial: [Sentiment Analysis in Python with Textblob](https://stackabuse.com/sentiment-analysis-in-python-with-textblob/)
* Doku: [spacytextblob](https://spacy.io/universe/project/spacy-textblob)

Prerequisites:
spacy, language model "en_core_web_sm", textblob, textblob corpus, spacytextblob (spacy pipeline component) installed

textblob: `pip3 install -U textblob`
textblob corpus: `python -m textblob.download_corpora`
spacytextblob: `pip install spacytextblob`

In [13]:
# load spacy
nlp = spacy.load("en_core_web_sm")

In [14]:
# show current spacy pipeline
nlp.components

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x127c25a7e20>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x127c242e400>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x127c24a6ba0>),
 ('senter', <spacy.pipeline.senter.SentenceRecognizer at 0x127c25a7fa0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x127c273a840>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x127c24c7840>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x127c24a6d60>)]

In [15]:
# add Textblob to pipeline
nlp.add_pipe("spacytextblob")

In [16]:
# check if added to pipe
nlp.components

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x127c25a7e20>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x127c242e400>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x127c24a6ba0>),
 ('senter', <spacy.pipeline.senter.SentenceRecognizer at 0x127c25a7fa0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x127c273a840>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x127c24c7840>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x127c24a6d60>),
 ('spacytextblob',
  <spacytextblob.spacytextblob.SpacyTextBlob at 0x127c2e6c5b0>)]

#### Sentiment Analysis with test text

In [17]:
# test text
text = "I love the lord of the rings. But I hate Sauron."

# doc of test text
doc = nlp(text)

In [18]:
# Sentiment Analysis: check polarity
# from -1.0 to 1.0
doc._.polarity

-0.15000000000000002

In [19]:
# Sentiment Analysis: check objectivity/subjectivity
doc._.subjectivity

0.75

In [20]:
# complete Sentiment Analysis
doc._.assessments
# Output: [(['love'], 0.5, 0.6, None)]
# ausschlaggebendes Wort, polarity, subjectivity

[(['love'], 0.5, 0.6, None), (['hate'], -0.8, 0.9, None)]

#### Sentiment Analysis zu Frodos Text 

In [21]:
# make doc of frodos dialog text
doc_frodo = nlp(frodo)

# frodo assessment
frodo_sa = doc_frodo._.assessments
frodo_sa

[(['late'], -0.3, 0.6, None),
 (['wonderful', '!'], 1.0, 1.0, None),
 (['whole'], 0.2, 0.4, None),
 (['half'], -0.16666666666666666, 0.16666666666666666, None),
 (['odd'], -0.16666666666666666, 0.25, None),
 (['mean'], -0.3125, 0.6875, None),
 (['more'], 0.5, 0.5, None),
 (['usual'], -0.25, 0.25, None),
 (['old'], 0.1, 0.2, None),
 (['right'], 0.2857142857142857, 0.5357142857142857, None),
 (['very'], 0.2, 0.3, None),
 (['unexpected'], 0.1, 1.0, None),
 (['glad'], 0.5, 1.0, None),
 (['back', '!', '!', '!', '!', '!'], 0.0, 0.0, None),
 (['long'], -0.05, 0.4, None),
 (['really'], 0.2, 0.2, None),
 (['only'], 0.0, 1.0, None),
 (['proud'], 0.8, 1.0, None),
 (['idiot'], -0.8, 0.8, None),
 (['good', '!'], 0.875, 0.6000000000000001, None),
 (['right'], 0.2857142857142857, 0.5357142857142857, None),
 (['hidden', '!', '!', '!', '!', '!', '!'],
  -0.6357828776041665,
  0.3333333333333333,
  None),
 (['safe'], 0.5, 0.5, None),
 (['easily', 'enough'], 0.0, 0.5, None),
 (['dangerous'], -0.6, 0.9, N

In [22]:
# Wie viele Sätze
sentences = frodo.split(".")
len(sentences)

138

In [26]:
# Wie viele Sätze nach NLP
len(frodo_sa)

62

In [24]:
# Warum verschiedene Ergebnisse bei Anzahl der Sätze???